In [64]:
import pandas as pd

from bs4 import BeautifulSoup
import os
import requests
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
from scipy import ndimage
from skimage.color import rgb2hsv
import skimage
from PIL import Image
import random
import cv2
import scipy.misc
from skimage import data, io
import pandas as pd

from skimage.filters.rank import entropy
from skimage.morphology import disk
from skimage.color import rgb2gray

In [11]:
pp = pd.read_csv('../data/picasso.csv')
hm = pd.read_csv('../data/henri_dic.csv')
from skimage import data, io

In [33]:
print('picasso rms start')
image_folder = os.path.join(os.pardir, 'data/picasso')
picasso_women_path = os.path.join(os.pardir, 'data/picasso_women.csv')
picasso_women = pd.read_csv(picasso_women_path)
picasso_women['RMS_contrast'] = picasso_women.apply(lambda x: RMS_CONTRAST(os.path.join(image_folder,x['Picturesource'])), axis = 1)

picasso rms start
..\data/picasso\images-works-3437_s.jpg
..\data/picasso\images-works-56_s.jpg
..\data/picasso\images-works-1513_s.jpg
..\data/picasso\images-works-3570_s.jpg
..\data/picasso\images-works-1459_s.jpg
..\data/picasso\images-works-151_s.jpg
..\data/picasso\images-works-1885_s.jpg
..\data/picasso\images-works-3040_s.jpg
..\data/picasso\images-works-102_s.jpg
..\data/picasso\images-works-128_s.jpg
..\data/picasso\images-works-1958_s.jpg
..\data/picasso\images-works-3960_s.jpg
..\data/picasso\images-works-61_s.jpg
..\data/picasso\images-works-3545_s.jpg
..\data/picasso\images-works-158_s.jpg
..\data/picasso\images-works-1651_s.jpg
..\data/picasso\images-works-2025_s.jpg
..\data/picasso\images-works-3941_s.jpg
..\data/picasso\images-works-3461_s.jpg
..\data/picasso\images-works-3447_s.jpg
..\data/picasso\images-works-174_s.jpg
..\data/picasso\images-works-1834_s.jpg
..\data/picasso\images-works-137_s.jpg
..\data/picasso\images-works-3351_s.jpg
..\data/picasso\images-works-173

In [35]:
picasso_women_path

'..\\data/picasso_women.csv'

In [70]:
def RMS_CONTRAST(imagefile):
    print(imagefile)
    img = cv2.imread(imagefile)
    oneD_pixels = np.concatenate(img, axis=0)

    brightness = 0
    ## average brightness
    for info in oneD_pixels:
        brightness += (0.2126 * info[0]) + (0.7152 * info[1]) + (0.0722 * info[2])
    average_brightness = brightness / len(oneD_pixels)  ## calculate average
    average_brightness /= 255  ## normalizing to 0 and 1

    rms = 0
    for info in oneD_pixels:
        brightness = (0.2126 * info[0]) + (0.7152 * info[1]) + (0.0722 * info[2])
        brightness /= 255  ## normalizing to 0 and 1
        rms += pow((brightness - average_brightness), 2);  ## calculate squared
    rms /= len(oneD_pixels)  ## calculate mean squared
    rms = pow(rms, 0.5)
    return rms

def average_entropy(imagefile):
    print(imagefile)
    img = cv2.imread(imagefile)
    gray_img = rgb2gray(img)
    return entropy(gray_img, disk(5)).mean()

def calc_stats_mat(picture):
    result=[]
    width= picture.shape[1]
    height=picture.shape[0]
    rgb_img = picture
    hsv_img = rgb2hsv(rgb_img)
    hue_img = hsv_img[:, :, 0]
    saturation_img = hsv_img[:,:, 1]
    value_img = hsv_img[:, :, 2]
    mean_hue = np.mean(hue_img, axis=(0,1))
    mean_saturation = np.mean(saturation_img, axis=(0,1))
    mean_brightness = np.mean(value_img)
    var_brightness = np.var(value_img)
    sobel_x = ndimage.sobel(value_img, axis=0, mode='constant')
    sobel_y = ndimage.sobel(value_img, axis=1, mode='constant')
    edge_image = np.hypot(sobel_x, sobel_y)
    sum_edge = np.sum(edge_image)/ edge_image.size
    result=[width,height,mean_hue,mean_saturation,mean_brightness,var_brightness,sum_edge,1]
    return result

def calc_stats_pic(picture):
    result=[]
    width= picture.shape[1]
    height=picture.shape[0]
    rgb_img = picture
    hsv_img = rgb2hsv(rgb_img)
    hue_img = hsv_img[:, :, 0]
    saturation_img = hsv_img[:,:, 1]
    value_img = hsv_img[:, :, 2]
    mean_hue = np.mean(hue_img, axis=(0,1))
    mean_saturation = np.mean(saturation_img, axis=(0,1))
    mean_brightness = np.mean(value_img)
    var_brightness = np.var(value_img)
    sobel_x = ndimage.sobel(value_img, axis=0, mode='constant')
    sobel_y = ndimage.sobel(value_img, axis=1, mode='constant')
    edge_image = np.hypot(sobel_x, sobel_y)
    sum_edge = np.sum(edge_image)/ edge_image.size
    result=[width,height,mean_hue,mean_saturation,mean_brightness,var_brightness,sum_edge,0]
    return result

In [71]:
# your code here
mat_dir = r'..\data\henri-matisse'
pic_dir = r'..\data\picasso'
db={}
for filename in os.listdir(mat_dir):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        temp_path=os.path.join(mat_dir, filename)
        temp_picture=io.imread(temp_path)
        temp_result=calc_stats_mat(temp_picture)
        db[filename]=temp_result
    else:
        continue
for filename in os.listdir(pic_dir):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        temp_path=os.path.join(pic_dir, filename)
        temp_picture=io.imread(temp_path)
        temp_result=calc_stats_pic(temp_picture)
        db[filename]=temp_result
    else:
        continue

In [72]:
dataframe= pd.DataFrame.from_dict(db, orient='index',
columns=['width', 'height', 'mean_hue', 'mean_saturation','mean_brightness','variance_brightness','sum_edge','matis'])
dataframe["Picturesource"] = dataframe.index
dataframe

,width,height,mean_hue,mean_saturation,mean_brightness,variance_brightness,sum_edge,matis,Picturesource
paintings-orient_107_thmb.jpg,150,119,0.245508,0.419425,0.699409,0.057506,0.574458,1,paintings-orient_107_thmb.jpg
paintings-orient_108_thmb.jpg,150,115,0.275711,0.289569,0.701977,0.095796,0.610242,1,paintings-orient_108_thmb.jpg
paintings-orient_109_thmb.jpg,134,182,0.183852,0.252422,0.685409,0.035527,0.478081,1,paintings-orient_109_thmb.jpg
paintings-orient_111_thmb.jpg,150,120,0.224027,0.345628,0.786914,0.028242,0.544646,1,paintings-orient_111_thmb.jpg
paintings-orient_113_thmb.jpg,150,121,0.170775,0.280903,0.686421,0.023599,0.477444,1,paintings-orient_113_thmb.jpg
paintings-orient_118_thmb.jpg,138,180,0.671918,0.280919,0.695900,0.037767,0.528205,1,paintings-orient_118_thmb.jpg
paintings-orient_123_thmb.jpg,139,179,0.162752,0.333270,0.752223,0.063703,0.561440,1,paintings-orient_123_thmb.jpg
paintings-orient_127_thmb.jpg,150,123,0.298297,0.504413,0.597549,0.066500,0.581258,1,paintings-orient_127_thmb.jpg
paintings-orient_129_thmb.jpg,150,120,0.465084,0.412532,0.490691,0.051506,0.476670,1,paintings-orient_129_thmb.jpg
paintings-orient_139_thmb.jpg,150,174,0.425217,0.325811,0.474165,0.048744,0.537306,1,paintings-orient_139_thmb.jpg


In [73]:
dataframe['RMS_contrast'] = dataframe.apply(lambda x: RMS_CONTRAST(os.path.join(mat_dir,x['Picturesource'])) if x['matis'] == 1 else RMS_CONTRAST(os.path.join(pic_dir,x['Picturesource'])), axis = 1)
dataframe['AVG_entropy'] = dataframe.apply(lambda x: average_entropy(os.path.join(mat_dir,x['Picturesource'])) if x['matis'] == 1 else average_entropy(os.path.join(pic_dir,x['Picturesource'])), axis = 1)

..\data\henri-matisse\paintings-orient_107_thmb.jpg
..\data\henri-matisse\paintings-orient_108_thmb.jpg
..\data\henri-matisse\paintings-orient_109_thmb.jpg
..\data\henri-matisse\paintings-orient_111_thmb.jpg
..\data\henri-matisse\paintings-orient_113_thmb.jpg
..\data\henri-matisse\paintings-orient_118_thmb.jpg
..\data\henri-matisse\paintings-orient_123_thmb.jpg
..\data\henri-matisse\paintings-orient_127_thmb.jpg
..\data\henri-matisse\paintings-orient_129_thmb.jpg
..\data\henri-matisse\paintings-orient_139_thmb.jpg
..\data\henri-matisse\paintings-orient_141_thmb.jpg
..\data\henri-matisse\paintings-orient_143_thmb.jpg
..\data\henri-matisse\paintings-orient_145_thmb.jpg
..\data\henri-matisse\paintings-orient_150_thmb.jpg
..\data\henri-matisse\paintings-orient_159_thmb.jpg
..\data\henri-matisse\paintings-orient_177_thmb.jpg
..\data\henri-matisse\paintings-orient_179_thmb.jpg
..\data\henri-matisse\paintings-orient_183_thmb.jpg
..\data\henri-matisse\paintings-orient_185_thmb.jpg
..\data\henr

..\data\picasso\images-works-1263_s.jpg
..\data\picasso\images-works-127_s.jpg
..\data\picasso\images-works-128_s.jpg
..\data\picasso\images-works-137_s.jpg
..\data\picasso\images-works-1450_s.jpg
..\data\picasso\images-works-1453_s.jpg
..\data\picasso\images-works-1459_s.jpg
..\data\picasso\images-works-145_s.jpg
..\data\picasso\images-works-1462_s.jpg
..\data\picasso\images-works-1471_s.jpg
..\data\picasso\images-works-1480_s.jpg
..\data\picasso\images-works-1482_s.jpg
..\data\picasso\images-works-1483_s.jpg
..\data\picasso\images-works-1486_s.jpg
..\data\picasso\images-works-148_s.jpg
..\data\picasso\images-works-1495_s.jpg
..\data\picasso\images-works-14_s.jpg
..\data\picasso\images-works-1513_s.jpg
..\data\picasso\images-works-1518_s.jpg
..\data\picasso\images-works-151_s.jpg
..\data\picasso\images-works-1525_s.jpg
..\data\picasso\images-works-1526_s.jpg
..\data\picasso\images-works-1528_s.jpg
..\data\picasso\images-works-1529_s.jpg
..\data\picasso\images-works-1535_s.jpg
..\data\

..\data\picasso\images-works-6_s.jpg
..\data\picasso\images-works-73_s.jpg
..\data\picasso\images-works-80_s.jpg
..\data\picasso\images-works-840_s.jpg
..\data\picasso\images-works-85_s.jpg
..\data\picasso\images-works-88_s.jpg
..\data\picasso\images-works-89_s.jpg
..\data\picasso\images-works-911_s.jpg
..\data\picasso\images-works-93_s.jpg
..\data\picasso\images-works-95_s.jpg
..\data\picasso\images-works-97_s.jpg
..\data\henri-matisse\paintings-orient_107_thmb.jpg
..\data\henri-matisse\paintings-orient_108_thmb.jpg
..\data\henri-matisse\paintings-orient_109_thmb.jpg
..\data\henri-matisse\paintings-orient_111_thmb.jpg
..\data\henri-matisse\paintings-orient_113_thmb.jpg
..\data\henri-matisse\paintings-orient_118_thmb.jpg
..\data\henri-matisse\paintings-orient_123_thmb.jpg
..\data\henri-matisse\paintings-orient_127_thmb.jpg
..\data\henri-matisse\paintings-orient_129_thmb.jpg
..\data\henri-matisse\paintings-orient_139_thmb.jpg
..\data\henri-matisse\paintings-orient_141_thmb.jpg
..\data\h

..\data\picasso\images-works-101_s.jpg
..\data\picasso\images-works-102_s.jpg
..\data\picasso\images-works-106_s.jpg
..\data\picasso\images-works-108_s.jpg
..\data\picasso\images-works-1107_s.jpg
..\data\picasso\images-works-117_s.jpg
..\data\picasso\images-works-120_s.jpg
..\data\picasso\images-works-122_s.jpg
..\data\picasso\images-works-1233_s.jpg
..\data\picasso\images-works-125_s.jpg
..\data\picasso\images-works-1263_s.jpg
..\data\picasso\images-works-127_s.jpg
..\data\picasso\images-works-128_s.jpg
..\data\picasso\images-works-137_s.jpg
..\data\picasso\images-works-1450_s.jpg
..\data\picasso\images-works-1453_s.jpg
..\data\picasso\images-works-1459_s.jpg
..\data\picasso\images-works-145_s.jpg
..\data\picasso\images-works-1462_s.jpg
..\data\picasso\images-works-1471_s.jpg
..\data\picasso\images-works-1480_s.jpg
..\data\picasso\images-works-1482_s.jpg
..\data\picasso\images-works-1483_s.jpg
..\data\picasso\images-works-1486_s.jpg
..\data\picasso\images-works-148_s.jpg
..\data\picas

..\data\picasso\images-works-41_s.jpg
..\data\picasso\images-works-44_s.jpg
..\data\picasso\images-works-51_s.jpg
..\data\picasso\images-works-55_s.jpg
..\data\picasso\images-works-56_s.jpg
..\data\picasso\images-works-57_s.jpg
..\data\picasso\images-works-61_s.jpg
..\data\picasso\images-works-62_s.jpg
..\data\picasso\images-works-64_s.jpg
..\data\picasso\images-works-6_s.jpg
..\data\picasso\images-works-73_s.jpg
..\data\picasso\images-works-80_s.jpg
..\data\picasso\images-works-840_s.jpg
..\data\picasso\images-works-85_s.jpg
..\data\picasso\images-works-88_s.jpg
..\data\picasso\images-works-89_s.jpg
..\data\picasso\images-works-911_s.jpg
..\data\picasso\images-works-93_s.jpg
..\data\picasso\images-works-95_s.jpg
..\data\picasso\images-works-97_s.jpg


In [74]:
dataframe

,width,height,mean_hue,mean_saturation,mean_brightness,variance_brightness,sum_edge,matis,Picturesource,RMS_contrast,AVG_entropy
paintings-orient_107_thmb.jpg,150,119,0.245508,0.419425,0.699409,0.057506,0.574458,1,paintings-orient_107_thmb.jpg,0.193308,5.052313
paintings-orient_108_thmb.jpg,150,115,0.275711,0.289569,0.701977,0.095796,0.610242,1,paintings-orient_108_thmb.jpg,0.275974,5.133200
paintings-orient_109_thmb.jpg,134,182,0.183852,0.252422,0.685409,0.035527,0.478081,1,paintings-orient_109_thmb.jpg,0.161583,4.809124
paintings-orient_111_thmb.jpg,150,120,0.224027,0.345628,0.786914,0.028242,0.544646,1,paintings-orient_111_thmb.jpg,0.174502,5.026028
paintings-orient_113_thmb.jpg,150,121,0.170775,0.280903,0.686421,0.023599,0.477444,1,paintings-orient_113_thmb.jpg,0.119690,5.111975
paintings-orient_118_thmb.jpg,138,180,0.671918,0.280919,0.695900,0.037767,0.528205,1,paintings-orient_118_thmb.jpg,0.191449,5.066507
paintings-orient_123_thmb.jpg,139,179,0.162752,0.333270,0.752223,0.063703,0.561440,1,paintings-orient_123_thmb.jpg,0.223050,4.703035
paintings-orient_127_thmb.jpg,150,123,0.298297,0.504413,0.597549,0.066500,0.581258,1,paintings-orient_127_thmb.jpg,0.204578,5.087020
paintings-orient_129_thmb.jpg,150,120,0.465084,0.412532,0.490691,0.051506,0.476670,1,paintings-orient_129_thmb.jpg,0.163515,4.789169
paintings-orient_139_thmb.jpg,150,174,0.425217,0.325811,0.474165,0.048744,0.537306,1,paintings-orient_139_thmb.jpg,0.184054,5.154482


In [75]:
dataframe.to_csv (r'../data/classifier.csv', index = False, header=True)